In [29]:
import tweepy
import os
import pandas as pd
import altair as alt
import re
import numpy as np

In [30]:
def get_polls_from_user(username, tweet_num = 10):
    '''
    Get list of poll ids for a given Twitter user

    Parameters
    ----------
    username : str
        username of the twitter user
    tweet_num: int
        number of the most recent tweets to be 
        default = 10
        maximum = 100, minimum = 5 per request by Twitter API
        
    Returns
    --------
        array of twitter ids

    Examples
    --------
    >>> get_polls_from_user('PollzOnTwitta')
    
    '''
    
    # Check argument validity
    if not(isinstance(username, str)):
        raise TypeError('Invalid argument type: username must be a string.')
    elif not(isinstance(tweet_num, int) and tweet_num >= 5 and tweet_num <= 100):
        raise TypeError('Invalid argument: input tweet_num must be >= 5 and <= 100.')

    # Twitter API credentials
    from dotenv import load_dotenv
    load_dotenv()
    bearer_token = os.environ.get("BEARER_TOKEN")
    client = tweepy.Client(bearer_token=bearer_token)
    
    # Get user_id from username
    users = client.get_users(usernames=username, user_fields=['id'])

    for user in users.data:  
        user_id = user.id

    # Get tweets specified by the requested user ID
    tweets = client.get_users_tweets(id=user_id, 
                                     max_results=tweet_num,
                                     expansions="attachments.poll_ids")

    # Get poll_ids from tweets if available
    poll_ids = []

    for tweet in tweets.data:
        if "attachments" in tweet.data.keys():
            attachments = tweet.data['attachments']
            poll_id = attachments['poll_ids']
            poll_ids.append(poll_id)
        else:
            pass  
    
    return poll_ids

In [31]:
#from tweepypoll import get_polls_from_user
from pytest import raises

def test_get_polls_from_user():
    """
    Test existing functionalities of get_polls_from_user()
    6 tests in total.
    """
    result = get_polls_from_user('PollzOnTwitta')

    # test output type
    assert type(result) == list

    # test output size (for an existing user)
    assert len(result) >= 0
    assert len(result) <= 100

    # test tweet_num argument, default = 10
    n = 20
    result = get_polls_from_user('PollzOnTwitta', tweet_num=n)
    assert len(result) <= n  